In [1]:
import requests

r = requests.get('https://www.mediafax.ro')
print(r.status_code)

200


In [2]:
print(r.content)

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ro" xml:lang="ro">\n\n    <head>\n\n        <meta name="google-site-verification" content="F84kaIgAlvwDvz__IYwu4zUHAExspfmJjzJ6ArraBh0" />\n        <meta name="google-site-verification" content="83qB9dMT3PdyVPeSXH8QpECV9DvGO-haV-FVBsQdLIo" />\n        <meta name="google-site-verification" content="llssVK3V5VYLD62XCUltG3dMTX62AlqwOWEJUGgNJmk" />\n\n        <meta http-equiv="Content-type" content="text/html;charset=UTF-8" />\n        <link rel="shortcut icon" href="/favicon.ico" />\n        <link rel="stylesheet" type="text/css" href="/style.css?v=10,11" media="screen" />\n\n         <script type="text/javascript">  </script>         <title>\xc5\x9etiri de ultima ora si ultimele \xc5\x9ftiri - \xc5\x9ftiri online - Mediafax\n</title>\n\n\t\t<meta property="fb:app_id" content="816679868905651"/>\n\t<meta property="fb:pages" content="109837159037642" />\n\t<meta property="og:site_name" content="Mediafax.ro" />\n\t<meta pro

In [3]:
import lxml.html

root = lxml.html.fromstring(r.content)
a_tags = root.xpath('//*[@id="content"]/div[3]/ul/li/p/a')
print(a_tags)

[<Element a at 0x10c33e580>, <Element a at 0x10c33e530>, <Element a at 0x10c33e4e0>, <Element a at 0x10c33e490>, <Element a at 0x10c33e440>, <Element a at 0x10c33ca00>, <Element a at 0x10c33c9b0>, <Element a at 0x10c33c960>, <Element a at 0x10c33c910>, <Element a at 0x10c33c8c0>, <Element a at 0x10c33c870>, <Element a at 0x10c33c820>, <Element a at 0x10c32fe80>, <Element a at 0x10c32ff70>, <Element a at 0x10c32fe30>, <Element a at 0x10c32fde0>, <Element a at 0x10c32fd90>]


In [4]:
url_list = []
for element in a_tags:
    href = element.attrib['href']
    if href.startswith('http://') or href.startswith('https://'): # if it's from a different website
        # url_list.append(href)
        pass
    elif href.startswith('/'):
        url_list.append('https://www.mediafax.ro' + href)
    else:
        url_list.append('https://www.mediafax.ro/' + href)
    
print(url_list)


['https://www.mediafax.ro/externe/germania-isi-recheama-ambasadorul-in-rusia-din-cauza-ingrijorarilor-legate-de-atacul-cibernetic-22369280', 'https://www.mediafax.ro/externe/razboiul-cu-drone-maritime-a-inceput-ce-fac-statele-unite-22369274', 'https://www.mediafax.ro/externe/macron-si-von-der-leyen-il-preseaza-pe-xi-al-chinei-cu-privire-la-comert-in-discutiile-de-la-paris-22369236', 'https://www.mediafax.ro/externe/israelul-incepe-evacuarea-unei-parti-din-rafah-hamas-denunta-o-escaladare-periculoasa-22369215', 'https://www.mediafax.ro/externe/uniunea-europeana-a-gasit-bani-pentru-inarmare-un-fond-400-de-miliarde-de-euro-ar-putea-fi-folosit-pentru-investitii-in-aparare-22369045', 'https://www.mediafax.ro/externe/vladimir-putin-ordona-exercitii-nucleare-ca-raspuns-la-amenintarile-lui-emmanuel-macron-22369195', 'https://www.mediafax.ro/externe/avertisment-al-serviciilor-secrete-europene-rusia-pregateste-activ-atentate-cu-bomba-sub-acoperire-atacuri-incendiare-si-distrugeri-ale-infrastruct

In [5]:
from collections import namedtuple

Item = namedtuple('Item', ['id', 'category', 'text'])
list_items = []

for url in url_list:
    article_id = url.split('-')[-1]
    category = url.split('/')[3]
    r = requests.get(url)
    url_root = lxml.html.fromstring(r.content)
    paragraphs = url_root.xpath('//*[@id="article_text_content"]/div[1]/p')
    text = ''
    for paragraph in paragraphs:
        text += paragraph.text_content().strip() + ' '
    item = Item(article_id, category, text)
    list_items.append(item)


In [6]:
# create the raw csv file 
import csv

with open('../../data/raw/romanian_news.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(Item._fields)
    for item in list_items:
        writer.writerow(item)

In [7]:
import spacy
nlp = spacy.load('ro_core_news_sm')

list_clean_items = []
for item in list_items:
    doc = nlp(item.text)
    ne = [ent.text for ent in doc.ents]
    clean_text = []
    for token in doc:
        if token.text in ne:
            clean_text.append("$NE$")
        else:
            clean_text.append(token.text)
    item = Item(item.id, item.category, ' '.join(clean_text))
    list_clean_items.append(item)

print(list_clean_items)

[Item(id='22369280', category='externe', text='„ Ministrul de externe l- a rechemat pe ambasadorul Graf $NE$ la $NE$ pentru consultări ” după ce $NE$ a atribuit o campanie de spionaj cibernetic APT $NE$ , un grup legat de agenţia de    informaţii militare $NE$ a Moscovei , a declarat purtătorul de cuvânt . Mişcarea este în conformitate cu protocolul diplomatic , iar $NE$ va rămâne la $NE$ timp de o săptămână înainte de a se întoarce la $NE$ , a adăugat purtătorul de cuvânt .   Ministerul $NE$ de Externe şi ambasada $NE$ la $NE$ nu au răspuns solicitărilor de comentarii . Acuzaţiile vin pe fondul unei anxietăţi crescute în $NE$ cu privire la presupuşii hackeri şi spioni ruşi de la invazia Moscovei în $NE$ în $NE$ şi în perioada premergătoare alegerilor    $NE$ . $NE$ a declarat că atacurile care au început în urmă cu doi ani au vizat social-democraţii aflaţi la guvernare în $NE$ , precum şi companii din sectoarele logistic , de apărare , aerospaţial    şi IT . Nu a oferit detalii despre

In [8]:
# create the clean csv file 
import csv

with open('../../data/clean/romanian_news.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(Item._fields)
    for item in list_clean_items:
        writer.writerow(item)